In [1]:
import pandas as pd
import numpy as np

# 데이터 로드 
df = pd.read_csv("data/merge_xy.csv") 

# 서비스 라벨별 평균 매출 계산
서비스별_평균_매출 = df.groupby('서비스_라벨')['당월_매출_금액'].mean()
서비스별_평균_매출 = 서비스별_평균_매출.reset_index()

# 상권 라벨별 평균 매출 계산
상권별_평균_매출 = df.groupby('상권_라벨')['당월_매출_금액'].mean()
상권별_평균_매출 = 상권별_평균_매출.reset_index()

# 행정동 라벨별 평균 매출 계산
행정동별_평균_매출 = df.groupby('행정동_라벨')['당월_매출_금액'].mean()
행정동별_평균_매출 = 행정동별_평균_매출.reset_index()

# 그룹별 평균 임대료 금액 계산
행정동별_평균_임대료 = df.groupby('행정동_라벨')['동별_임대료'].mean()

In [2]:
import pandas as pd
import numpy as np

def calculate_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹, level="서비스_라벨"):
   
    # 그룹별 가중치 선택
    if level == "서비스_라벨":
        weights_df = pd.read_csv("data/group_average_weights_서비스_라벨.csv", index_col=0)
    elif level == "상권_라벨":
        weights_df = pd.read_csv("data/group_average_weights_상권_라벨.csv", index_col=0)
    elif level == "행정동_라벨":
        weights_df = pd.read_csv("data/group_average_weights_행정동_라벨.csv", index_col=0)
    else:
        raise ValueError("올바른 그룹 기준을 입력하세요. (서비스_라벨 / 상권_라벨 / 행정동_라벨)")

   
    weights_df.index = weights_df.index.astype(str)
    그룹 = str(그룹)  # 그룹 변수를 문자열로 변환

    if 그룹 not in weights_df.index:
        print(f"그룹 '{그룹}'의 데이터가 없습니다. 기본 가중치를 사용합니다.")
        weights = np.array([-0.3, 0.2, -0.15, 0.25, 0.1, -0.2, 0.15, 0.1])  # 기본 가중치
    else:
        weights = weights_df.loc[그룹].values
    
    if len(weights) < 7:
        raise ValueError(f"가중치 개수가 부족합니다: {weights}")
    
    
    if level == "서비스_라벨":
        그룹별_평균_매출 = 서비스별_평균_매출.set_index("서비스_라벨")["당월_매출_금액"]
    elif level == "상권_라벨":
        그룹별_평균_매출 = 상권별_평균_매출.set_index("상권_라벨")["당월_매출_금액"]
    else:
        그룹별_평균_매출 = 행정동별_평균_매출.set_index("행정동_라벨")["당월_매출_금액"]

    # 인덱스를 문자열로 변환 후 그룹별 평균 매출 가져오기
    그룹별_평균_매출.index = 그룹별_평균_매출.index.astype(str)
    그룹_평균_매출 = 그룹별_평균_매출.get(그룹, 1000000)
    print(f"그룹 '{그룹}'의 평균 매출: {그룹_평균_매출}")

    # 매출비율 계산 
    매출비율 = np.clip(매출비율 / 그룹_평균_매출, 0, 1)

    # 행정동별 평균 임대료 가져오기
    행정동별_평균_임대료.index = 행정동별_평균_임대료.index.astype(str)
    그룹_평균_임대료 = 행정동별_평균_임대료.get(그룹, 1000000)
    print(f"그룹 '{그룹}'의 평균 임대료: {그룹_평균_임대료}")

    # 임대료 비율 계산
    임대료비율 = np.clip(임대료비율 / 그룹_평균_임대료, 0, 1)

    # 생존 확률 계산
    W1, W2, W3, W4, W5, W6, W8 = weights[:7]
    S = (W1 * (1 - 폐업률) +
         W2 * 개업률 +
         W3 * min(유사점포수 / 100, 1) +
         W4 * 매출비율 +        #(매출비율 / 1e9)
         W5 * 유동인구 / 10000000 +
         W6 * 상주인구 / 1000000 +
         W8 * (1 - 임대료비율))
    
    S = np.clip(S, 0, 1)
    survival_rate = np.clip(S * 100, 0, 100)
    
    return round(survival_rate, 2)

# 이건 그 사용자가 상권-업종 선택하면 알아서 불러올 수 있도록 하고 싶었는데 아직 덜 했어요
폐업률 = 0.04 
개업률 = 0.06  
유사점포수 = 269 
매출비율 = 0.5  
유동인구 = 1133819  
상주인구 = 1668 
임대료비율 = 0.9  # 임대료 부담 비율

# 특정 그룹의 평균 가중치 적용하여 생존 확률 계산
그룹명 = "9"
level = "서비스_라벨"

survival_rate = calculate_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹=그룹명, level=level)
print(f"{그룹명} 업종의 100일 생존 확률: {survival_rate}%")


그룹 '9'의 평균 매출: 579026985.005007
그룹 '9'의 평균 임대료: 103168.0
9 업종의 100일 생존 확률: 39.77%


In [3]:
def calculate_combined_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹, level="서비스_라벨"):
    
    # 서비스 라벨 생존 확률 계산
    service_survival_rate = calculate_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹, level="서비스_라벨")
    print(f"서비스 라벨의 생존 확률: {service_survival_rate}%")
    
    # 상권 라벨 생존 확률 계산
    market_survival_rate = calculate_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹, level="상권_라벨")
    print(f"상권 라벨의 생존 확률: {market_survival_rate}%")
    
    # 행정동 라벨 생존 확률 계산
    district_survival_rate = calculate_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹, level="행정동_라벨")
    print(f"행정동 라벨의 생존 확률: {district_survival_rate}%")
    
    # 가중치는 임의로 한 거예요,,
    service_weight = 0.5
    market_weight = 0.3
    district_weight = 0.2
    
    combined_survival_rate = (service_weight * service_survival_rate +
                              market_weight * market_survival_rate +
                              district_weight * district_survival_rate)
    
    print(f"최종 종합 생존 확률: {combined_survival_rate}%")
    
    return round(combined_survival_rate, 2)

# 이건 그 사용자가 상권-업종 선택하면 알아서 불러올 수 있도록 하고 싶었는데 아직 덜 했어요 22 
폐업률 = 0.04 
개업률 = 0.06  
유사점포수 = 269 
매출비율 = 0.5 
유동인구 = 1133819  
상주인구 = 1668 
임대료비율 = 0.2  

# 특정 그룹의 평균 가중치 적용하여 생존 확률 계산
그룹명 = "9"
level = "서비스_라벨"

combined_survival_rate = calculate_combined_survival_rate(폐업률, 개업률, 유사점포수, 매출비율, 유동인구, 상주인구, 임대료비율, 그룹=그룹명, level=level)

print(f"{그룹명} 업종의 100일 종합 생존 확률: {combined_survival_rate}%")


그룹 '9'의 평균 매출: 579026985.005007
그룹 '9'의 평균 임대료: 103168.0
서비스 라벨의 생존 확률: 39.77%
그룹 '9'의 평균 매출: 220853514.09375
그룹 '9'의 평균 임대료: 103168.0
상권 라벨의 생존 확률: 30.05%
그룹 '9'의 평균 매출: 436986585.96219933
그룹 '9'의 평균 임대료: 103168.0
행정동 라벨의 생존 확률: 33.49%
최종 종합 생존 확률: 35.598%
9 업종의 100일 종합 생존 확률: 35.6%
